# GeoJSON Construction

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import csv 
import os 
import json 
from tqdm import tqdm_notebook
import pandas as pd

## Create breeding site GeoJSON information from Raw JSON files 

In [2]:
def insert_data(SUM, Feature, row):
     
    if(len(row['breading_boxes']) != 0):
        sum_each = 0
        for cls in row['count']:
            sum_each += row['count'][cls]

        Feature['properties']['brd_sites'][str(row['angle'])] = {'count': row['count']}
        Feature['properties']['brd_sites'][str(row['angle'])]['sum_each'] = sum_each
        SUM += sum_each
    
    return SUM, Feature

def run(directory):
    SUM = 0
    base_coor = ''
    check_coor = []
    GeoJSON = {} 
    GeoJSON['type'] = "FeatureCollection" 
    GeoJSON['features'] = []
    Feature = {}
    
    json_list = os.listdir(os.path.join(directory, 'json'))
    json_list.sort()
    
    for json_file in tqdm_notebook(json_list):
        
        with open(os.path.join(directory, 'json', json_file)) as file:
            row = json.load(file)
            
        lat, lng = row['coordinate']
        coor = str(lat)+'_'+str(lng)
        
        if(base_coor == coor):
            SUM, Feature = insert_data(SUM, Feature, row)
        else:
            base_coor = coor
            if(SUM != 0):
                Feature['properties']['sum'] = SUM
                GeoJSON['features'].append(Feature)
                SUM = 0
            
            
            Feature = { 
                'type': "Feature", 
                   'geometry':{ 
                       'type':"Point",
                       'coordinates':[float(lng),float(lat)] 
                   },
                'properties':{
                    'image_name': coor, 
                    'brd_sites':{},
                    'date':{
                        'month':row['date']['month'], 
                        'year':row['date']['year']
                    },
                    'province':row['province'],
                    'district':row['district'],
                    'subdist':row['subdist']
                  }
              }
            SUM, Feature = insert_data(SUM, Feature, row)
    
    # last row will not be detected in for loop
    if(SUM != 0):
        Feature['properties']['sum'] = SUM
        GeoJSON['features'].append(Feature)
        SUM = 0

    with open(os.path.join(directory,'brd_sites.js'), 'w') as f: 
        f.write('brd_sites('+json.dumps(GeoJSON, ensure_ascii=False, sort_keys=True, indent=4, separators=(',', ': '))+')')
        
    with open(os.path.join(directory,'brd_sites.geojson'), 'w') as f: 
        f.write(json.dumps(GeoJSON, ensure_ascii=False, sort_keys=True, indent=4, separators=(',', ': ')))

## Get the list of directories or subdistricts
That contains path_to_directory, province, district, and subdistrict 

In [3]:
directories = []

province = 'กรุงเทพมหานคร'
DIR = '../data/GSV'

districts = os.listdir(os.path.join(DIR, province))
for district in districts:
    subdists = os.listdir(os.path.join(DIR, province, district))
    for subdist in subdists:
        directories.append([DIR, province, district, subdist])

print(directories[0])

['../data/GSV', 'กรุงเทพมหานคร', 'เขตดุสิต', 'แขวงสวนจิตรลดา']


## Iterating though directories

In [4]:
last = 0
for k, directory in enumerate(directories[last:]):
    DIR, province, district, subdist = directory
    directory = os.path.join(DIR, province, district, subdist)
    
    image_path = os.path.join(directory, 'original')
    if not os.path.exists(image_path): continue
    
    FILES = os.listdir(image_path)
    
    print('Index:',k+last,':\tDirectory:',directory, '\tTotal:',len(FILES),'images')
    run(directory)

Index: 0 :	Directory: ../data/GSV/กรุงเทพมหานคร/เขตดุสิต/แขวงสวนจิตรลดา 	Total: 1 images


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))